In [38]:
from selenium import webdriver
import selenium.common.exceptions
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [39]:
def get_whitelist_from_user():
    # retrieve user's whitelist
    # elements of whitelist : device set, region list, city list
    whitelist = list()
    device = 0    
    global region_list
    
    region_list = [
        "서울",
        "경기/인천",
        "충청/강원",
        "경상/제주",
        "전라"
    ]
    
    while True:
        try:
            # 1) select a devices 
            print("")
            print("-"*20)
            print("대여를 희망하는 기종을 하나 선택해주세요")
            print("1. Galaxy Z Flip 3")
            print("2. Galaxy Z Flip 3, Watch 4")
            print("3. Galaxy Z Flip 3, Watch 4 Classic")
            print("4. Galaxy Z Fold 3")
            print("5. Galaxy Z Fold 3, Watch 4")
            print("6. Galaxy Z Fold 3, Watch 4 Classic")
            print("Q. 종료")
            print("-"*20)
            print("")
            
            decision = input("선택 : ")
            
            if (decision == "Q"):
                return None
            
            decision = int(decision)
            
            if (decision < 1 or decision > 6):
                raise ValueError
                
            whitelist.append(decision - 1)
            
            decision = str()
            
            # 2) select regions where you are willing to go
            print("")
            print("-"*20)
            print("매장이 위치한 지역들 중 희망하는 지역을 선택해주세요")
            print("여러 지역을 선택할 수 있습니다. 각 선택지는 공백으로 구분됩니다.")
            print("1. 서울")
            print("2. 경기/인천")
            print("3. 충청/강원")
            print("4. 경상/제주")
            print("5. 전라")
            print("Q. 종료")
            print("-"*20)
            print("")            
            
            # convert integer type element to string type element
            decision = input("선택 : ")
            
            if (decision in "Q"):
                return None
            
            decision = map(int, decision.split(" "))
            
            decision = list(map(lambda x: region_list[x - 1] if x >= 1 and x <= 5 else None, decision))
            
            whitelist.append(decision)

            decision = str()
            
            # 3) specify city where you are really willing to go
            print("")
            print("-"*20)
            print("우선적으로 방문하고자 하는 시를 입력하세요")
            print("여러 곳 입력 가능하며, 공백으로 각 시를 구분합니다.")
            print("Q를 입력 시 종료합니다.")
            print("-"*20)
            print("")
            
            decision = input("선택 : ")
            
            if (decision == "Q"):
                return None
            
            whitelist.append(decision.split(" "))
        
            return whitelist
        
        except:
            print("입력 값을 확인해주세요.")
            break #debug
            #continue

In [40]:
# Open chromedriver where is your project directory and make it go to the desired website
def get_web_driver(file_path):
    driver = webdriver.Chrome(file_path)
    driver.get(step1_url)
    return driver # returns the webdriver instance

In [41]:
# On the step 1 page, click "Galaxy Z Flip 3" banner and enter that step 2 page
def go_to_desired_item_page(driver, desired_item_index):
    links = driver.find_elements_by_css_selector(".btn > img")

    # find a link about desired item among the button elements
    link = links[desired_item_index]
    link.click()
    
    # when alert screen is shown, click OK to go to the step 2 page.
    try:
        alert = driver.switch_to.alert
        alert.accept()   # go to step 2 page

    except selenium.common.exceptions.NoAlertPresentException:
        print('Unable to get alert element on your driver.')
    
    finally:
        return None

In [57]:
def get_info_of_stores(driver, whitelist):
    try:
        # Check whether the current url is step 2 web page,
        # then, get the all of links of regions
        if driver.current_url == step2_url:
            region_whitelist, city_whitelist = whitelist
            store_info_lists = list()
        else:
            raise ValueError
        
        region_link_list = driver.find_elements_by_css_selector(".location > .btn")
        
        for region in region_list:
            if region in region_whitelist:
                driver.find_element_by_css_selector(f".location > .btn:nth-child({region_list.index(region) + 1})").click()
                
            
        # 2) find city matches the context of whitelist
            reserve_shops = driver.find_elements_by_css_selector(".store_list > .list > li")
    
            # Get info about stores what you want to go
            for reserve_shop in reserve_shops:
                store_info_list = list()

                store_name = reserve_shop.find_element_by_css_selector("dl > dt").get_attribute("innerText")
                store_address = reserve_shop.find_element_by_css_selector("dl > dd").get_attribute("innerText")
                store_remaining = reserve_shop.find_element_by_css_selector(".status > strong").get_attribute("innerText")[:-1]

                store_info_list.append(store_name)
                store_info_list.append(store_address)
                store_info_list.append(store_remaining)

                store_info_lists.append(store_info_list)           

        return store_info_lists
            
    except:
        print("An error occurred")
        return None

In [ ]:
def await_until_vacancy(driver, store_info):
'''
# for finding activated button
select_button = reserve_shop.find_element_by_css_selector(".lnk > a:last-child")
print(select_button.get_attribute("innerText"))

# for noticing any vacant stores
vacant_store = reserve_shop.find_element_by_css_selector("p.status > strong").get_attribute("innerText")
if (vacant_store != "0대"):
    print(datetime.now(), "에 발견!")
''' 
    pass

In [58]:
file_path = "C://Users//deblu//PycharmProjects//reserve_ToGo//chromedriver.exe"
global step1_url
global step2_url

step1_url = "https://www.galaxytogo.co.kr/reservation/step1"
step2_url = "https://www.galaxytogo.co.kr/reservation/step2"

whitelist = get_whitelist_from_user()

if whitelist is not None:
    driver = get_web_driver(file_path)
    go_to_desired_item_page(driver, whitelist[0])
    store_info = get_info_of_stores(driver, whitelist[1:])
    print(store_info) # debug
    
    await_until_vacancy(driver, store_info)

    driver.close()


--------------------
대여를 희망하는 기종을 하나 선택해주세요
1. Galaxy Z Flip 3
2. Galaxy Z Flip 3, Watch 4
3. Galaxy Z Flip 3, Watch 4 Classic
4. Galaxy Z Fold 3
5. Galaxy Z Fold 3, Watch 4
6. Galaxy Z Fold 3, Watch 4 Classic
Q. 종료
--------------------



선택 :  1



--------------------
매장이 위치한 지역들 중 희망하는 지역을 선택해주세요
여러 지역을 선택할 수 있습니다. 각 선택지는 공백으로 구분됩니다.
1. 서울
2. 경기/인천
3. 충청/강원
4. 경상/제주
5. 전라
Q. 종료
--------------------



선택 :  1 2 3



--------------------
우선적으로 방문하고자 하는 시를 입력하세요
여러 곳 입력 가능하며, 공백으로 각 시를 구분합니다.
Q를 입력 시 종료합니다.
--------------------



선택 :  용인


[['모바일스토어 하이마트 잠실점', '서울특별시 송파구 올림픽로 240 롯데마트 잠실점 1층', '0'], ['모바일스토어 하이마트 월드타워점', '서울특별시 송파구 올림픽로 300 롯데월드몰 3층', '0'], ['디지털프라자 삼성대치본점', '서울특별시 강남구 영동대로 237 1F', '0'], ['디지털프라자 강남본점', '서울특별시 강남구 도산대로 336 2F', '0'], ['디지털프라자 도곡점', '서울특별시 강남구 언주로 201 에스케이리더스뷰 1F', '0'], ['디지털프라자 서초본점', '서울특별시 서초구 서초대로52길 12(서초동) 1F', '0'], ['삼성딜라이트샵', '서울특별시 서초구 서초대로74길 11 삼성전자빌딩 지하1층', '0'], ['디지털프라자 홍대본점', '서울특별시 마포구 양화로 171 1F', '0'], ['디지털프라자 강서본점', '서울특별시 강서구 공항대로 571 1F', '0'], ['모바일스토어 일렉트로마트 영등포점', '서울특별시 영등포구 영중로 15 타임스퀘어 B1 일렉트로마트', '0'], ['디지털프라자 강북본점', '서울특별시 성북구 월계로 30 1F', '0'], ['KT플라자 강남역점', '서울특별시 강남구 테헤란로 120 1층', '0'], ['디지털프라자 일산본점', '경기도 고양시 일산동구 중앙로 1219 1F', '0'], ['디지털프라자 용인구성본점', '경기도 용인시 기흥구 용구대로 2458 1F', '0'], ['디지털프라자 부천중동본점', '경기도 부천시 길주로 254 1F', '0'], ['디지털프라자 안양본점', '경기도 안양시 동안구 시민대로 203 1F', '0'], ['모바일스토어 AK플라자 분당점', '경기도 성남시 분당구 황새울로360번길 42 AK플라자 1층', '0'], ['모바일스토어 현대백화점판교점', '경기도 성남시 분당구 판교역로146번길 20현대백화점 판교점 5층', '0'], ['모바일스토어 갤러리아광교점', '경기도 수원시 영통구 광교중앙로 124 6층'